In [1]:
from utils.data_preprocess_utils import get_config, get_all_files, get_all_subdirectories
from easydict import EasyDict
from utils.dataset import DenmarkDataset

In [2]:
cfg = EasyDict(get_config('./cfg/dataset_cfg.yaml'))
dataset = DenmarkDataset(cfg)
# data_dic is in the format of {data_type: data_array of specific type}, data_array is in the format of (num_samples, num_waypoints, num_features)
print(f'access one day data with time interval 4H')
day_data_dic = dataset._get_day_data(day='2019-01-01', time_interval='4H')
print(f'day_data for Total shape -> (interval, wp, features) {day_data_dic["Total"].shape}\n')

print(f'access data from 2019-01-01 to 2019-03-31 with time interval 4H')
range_data_dic = dataset.get_data_by_range(start_date='2019-01-01', end_date='2019-03-31', time_interval='4H')
print(f'range_data_dic for Total shape -> (interval, wp, features) {range_data_dic["Total"].shape}\n')

print(f'convert data_dic to data_arr')
vessel_types = ['Passenger', 'Tanker', 'Cargo', 'Other', 'Total']
range_data_arr, type_index_dic = dataset.dic2array(result_dic=range_data_dic, vessel_types=vessel_types, time_interval='4H')
print(f'range_data_arr shape -> (day, interval, wp, vessel_types, features): {range_data_arr.shape}\n')

print(f'access data of specific vessel type')
range_data_arr_tanker = range_data_arr[:, :, :, type_index_dic['Tanker'], :]
print(f'range_data_arr_tanker shape -> (day, interval, wp, features): {range_data_arr_tanker.shape}\n')

access one day data with time interval 4H
day_data for Total shape -> (interval, wp, features) (6, 92, 4)

access data from 2019-01-01 to 2019-03-31 with time interval 4H
range_data_dic for Total shape -> (interval, wp, features) (540, 92, 4)

convert data_dic to data_arr
range_data_arr shape -> (day, interval, wp, vessel_types, features): (90, 6, 92, 5, 4)

access data of specific vessel type
range_data_arr_tanker shape -> (day, interval, wp, features): (90, 6, 92, 4)



In [3]:
dataset.convert2PEMS()

In [9]:
import pandas as pd
df = pd.read_csv('/root/GIS/ais_PEMS/AISFlow.rel')


0       False
1       False
2       False
3       False
4       False
        ...  
8367    False
8368    False
8369    False
8370     True
8371     True
Name: weight, Length: 8372, dtype: bool